In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("arxiv_dataset")

In [ ]:
!pip install kaggle

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "marshmellow77"
os.environ['KAGGLE_KEY'] = "5b2c4fe92c68d358ef8ad52ece20445f"

In [ ]:
import kaggle
kaggle.api.authenticate()

In [ ]:
kaggle.api.dataset_list(search="arxiv")

In [ ]:
kaggle.api.dataset_download_files('Cornell-University/arxiv', path=".")

In [ ]:
!unzip arxiv.zip

In [ ]:
dataset = load_dataset("arxiv_dataset", data_dir='.')

In [ ]:
# The first 10% of `train` split.
dataset = load_dataset("arxiv_dataset", data_dir='.', split='train', ignore_verifications=True)

In [ ]:
dataset

In [ ]:
dataset = dataset.shuffle(seed=42)
dataset = dataset.select(range(25000))
dataset

In [ ]:
import pandas as pd
df = pd.DataFrame(dataset)

In [ ]:
 # only keep columns that are required
df = df[['abstract', 'title']]
df = df.rename(columns={"abstract": "text", "title": "summary"})

In [ ]:
df = df.replace(r'\n',' ', regex=True)

In [ ]:
pd.options.display.max_colwidth = 100

In [ ]:
df.head()

In [ ]:
cutoff_summary = 5
cutoff_text = 20
df = df[(df['summary'].apply(lambda x: len(x.split()) >= cutoff_summary)) & (df['text'].apply(lambda x: len(x.split()) >= cutoff_text))]

In [ ]:
len(df)

In [ ]:
df = df.sample(20000, random_state=43)

In [ ]:
import numpy as np
# split the dataset into train, val, and test
df_train, df_val, df_test = np.split(df.sample(frac=1), [int(0.8*len(df)), int((0.9)*len(df))])

In [ ]:
df_train.to_csv('data/train.csv', index=False)
df_val.to_csv('data/val.csv', index=False)
df_test.to_csv('data/test.csv', index=False)

In [ ]:
import sagemaker
sess = sagemaker.Session()
bucket = sess.default_bucket()

In [ ]:
!aws s3 cp data/train.csv s3://$bucket/summarization/data/train.csv
!aws s3 cp data/val.csv s3://$bucket/summarization/data/val.csv
!aws s3 cp data/test.csv s3://$bucket/summarization/data/test.csv